In [5]:
import igraph as ig
from typing import Dict
import matplotlib.pyplot as plt
import overpy

In [6]:
api = overpy.Overpass()
result = api.query("nwr(56.3344, -2.8153, 56.3434, -2.7836); out;")

In [15]:
type(result.nodes[0].id)

int